# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [ ]:
%%capture
#!pip install wandb
!pip install pandas

### Step 1️⃣: Import W&B and Login

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 

#%matplotlib inline

In [ ]:
import wandb
from wandb.keras import WandbCallback

#wandb.init()  # defaults are over-ridden during the sweep


#wandb.login()

In [ ]:

fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

Read data in

# 👩‍🍳 Prepare Dataset

In [ ]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


# Build simple Model

In [ ]:
AoModel = keras.Sequential([
     keras.layers.InputLayer(19, name="digits"),
     keras.layers.Dense(2000, activation="relu"),
     keras.layers.Dense(1050, activation="relu"),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(9, activation="linear", name="predictions"),

])
AoModel.summary()

In [ ]:
batch_size = 128
epochs = 200


    # optimizer='sgd',
    # loss='mse',
    # metrics=[tf.keras.metrics.RootMeanSquaredError()])



AoModel.compile(loss= keras.losses.MeanSquaredError(), optimizer="adam", metrics= [tf.keras.metrics.RootMeanSquaredError()])
#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,  callbacks=[WandbCallback()])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

# 🧠 Define the wandb sweep

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoKerasStudio.ipynb"

sweep_config = {
    "method": "random", # try grid or random
    "metric": {
        "name": "val_root_mean_squared_error",
        "goal": "minimize"
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
      "min": 6,
      "max": 10
        },


    "learning_rate" :{
        "values": [0.04954]
        },        
    "subsample": {
        "values": [0.4372]
    },
    "colsample_bynode": {
        "values": [0.7597]
    },    
    "batch_size": {
        "values": [64, 128]
    },
    "epochs": {
        "values": [2, 4]
    },    
    "reg_lambda": {
        "values": [0.5222]
    }                
    }
}

In [ ]:
## Define training loop

In [ ]:
def train():
  config_defaults = {
  "booster": "gbtree",
  "max_depth": 3,
  "learning_rate": 0.1,
  "subsample": 1,
  "seed": 117,
  "test_size": 0.33,
}
  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  config = wandb.config
  AoModel.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epochs, verbose = 2,validation_split = 0.2 , callbacks=[WandbCallback()])
  #wandb.log({"error": "val_root_mean_squared_error"})

## Initialize sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = "uwe-sterr",project="KerasAoSweep")
wandb.run

## Run sweep

In [ ]:
wandb.agent(sweep_id, train, count=4)